# CodeKata Notebook

Here I will do some of the tasks stated on http://codekata.com/ just for some personal coding practice

## Kata02: Karate Chop

The task was to implement a binary search routine using different techniques

In [17]:
def assert_int(k,x):
    """Checks if integer k is in list x and returns corresponding index if true, and -1 else"""
    l = len(x)
    i=-1
    if k in x:
        i=0
        while l >= 1:
            if k in x[i:i+l-l//2]:
                l = l//2
            else:
                l = l - l//2
                i = i + l - l//2
    return(i)


In [18]:
z = [2,4,3,2,7,6,1]

In [19]:
for i in range(10):
    print(f"for {i} assert_int returns {assert_int(i,z)}")

for 0 assert_int returns -1
for 1 assert_int returns 6
for 2 assert_int returns 0
for 3 assert_int returns 2
for 4 assert_int returns 1
for 5 assert_int returns -1
for 6 assert_int returns 5
for 7 assert_int returns 4
for 8 assert_int returns -1
for 9 assert_int returns -1


Now, lets do the same in an iterative approach

In [20]:
def iter(k,x):
    if k not in x:
        return -1
    l = len(x)
    if l == 1:
        return 0
    else:
        if k in x[:l-l//2]:
            i = iter(k,x[:l-l//2])         
            print(f"i ist {i} und x ist {x[:l-l//2]}")
            return i
        else:
            i = iter(k,x[l-l//2:]) 
            print(f"i ist {i} und x ist {x[l-l//2:]}")
            return i + l-l//2            


In [21]:
for i in range(10):
    print(f"for {i} assert_int returns {assert_int(i,z)}")

for 0 assert_int returns -1
for 1 assert_int returns 6
for 2 assert_int returns 0
for 3 assert_int returns 2
for 4 assert_int returns 1
for 5 assert_int returns -1
for 6 assert_int returns 5
for 7 assert_int returns 4
for 8 assert_int returns -1
for 9 assert_int returns -1


## Kata04: Data Munging

Here the exercise was to write two functions basically evaluating, where the difference of two columns from a table is minimal.

In [22]:
import csv
import numpy as np
from numpy import argmax
import pandas as pd


def min_delta(path):
    dat = [i.split() for i in open(path).readlines()]
    dat2 = dat[2:-1]

    df = pd.DataFrame()
    df['day'] = [int(i[0]) for i in dat2]
    df['max_T'] = [int(i[1].strip('*')) for i in dat2]
    df['min_T'] = [int(i[2].strip('*')) for i in dat2]
    df['delta_T'] = df['max_T'] - df['min_T']
    i = df['delta_T'].argmin()
    
    return df['day'][i] 


min_delta("./weather.dat")


14

In [23]:
def min_team(path):
    dat = [i.strip().split() for i in open(path).readlines()]
    del dat[0]
    del dat[17]

    df = pd.DataFrame()
    df['team'] = [i[1] for i in dat]
    df['F'] = [int(i[6]) for i in dat]
    df['A'] = [int(i[8]) for i in dat]
    df['delta_G'] = df['F'] - df['A']
    i = df['delta_G'].argmin()
    
    print(df['team'][i])


min_team("./football.dat")

Leicester


In a second step, we factor out as much common code as possible.

In [24]:
def weird_fct(path, lines_to_delete, target, upper, lower):
    """reads a dat-file from path and calculates the value of target, where upper - lower is minimal"""
    dat = [i.strip().split() for i in open(path).readlines()]
    lines_to_delete.sort(reverse=True)
    for l in lines_to_delete:
        del dat[l]
    df = pd.DataFrame(dat)
    df['delta'] = df[upper].transform(lambda x: int(x.strip('*'))) - df[lower].transform(lambda x: int(x.strip('*')))
    i = df['delta'].argmin()
    return df[target][i]

In [25]:
weird_fct("./football.dat", [0,18], 1, 6, 8)

'Leicester'

In [26]:
weird_fct("./weather.dat", [0,1,32], 0, 1, 2)

'14'

## Kata05: Bloom Filters

In [27]:
from bitarray import bitarray
from bitarray.util import ba2int, int2ba
import mmh3
import timeit

In [28]:
class BloomFilter():
    def __init__(self, word_list, number_hash_fct = 3, length_bitarray = 2**20):
        self.word_list = word_list
        self.number_hash_fct = number_hash_fct
        self.length_bitarray = length_bitarray
        self.BA = int2ba(0, length = length_bitarray)

        for w in word_list:
            for i in range(number_hash_fct):
                self.BA[mmh3.hash(w, seed = i) % length_bitarray] = 1

    def is_in(self, word):
        return all([self.BA[mmh3.hash(word, seed = i) % self.length_bitarray] == 1 for i in range(self.number_hash_fct)])
        


In [29]:
words = [i.strip() for i in open("./wordlist.txt").readlines()]

In [30]:
BF = BloomFilter(words)

In [31]:
%timeit BF.is_in('kekson')
%timeit 'kekson' in words

2.1 µs ± 42.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.66 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


No, lets check how many false positives we get

In [32]:
import string
import random

def word_gen(length = 5):
    """Creates a random 'word'"""
    w = ''
    for i in range(length):
        w += random.choice(string.ascii_letters)
    return w

In [33]:
i = 0
for j in range(10000):
    w = word_gen()
    if BF.is_in(w) and w not in words:
        i += 1

print(i)

2475


Quite some. Lets try a bigger array

In [34]:
BF_2 = BloomFilter(words, 3, 2**30)

i = 0
for j in range(100000):
    w = word_gen()
    if BF_2.is_in(w) and w not in words:
        i += 1

print(i)

0


## Kata06: Anagrams

Yet to be done

In [35]:
# word_sets = [(letter for letter in word) for word in words]
# set_of_sets = (word_sets)

# non_unique = [word for word in word_sets if word not in set_of_sets]
